### 데이터프레임의 결합 
- 유니언 결합 
    - 단순하게 행을 결합하거나 열을 결합하는 방식
    - pandas에 내장된 concat() 함수를 이용
        - axis 매개변수
            - 0(rows) : 행을 결합 (기본값)
            - 1(columns) : 열을 결합 
        - ignore_index 매개변수
            - False(기본값) : 결합되는 인덱스들을 그대로 보존
            - True : 결합되는 인덱스를 초기화
    - 다수의 데이터프레임을 한번에 결합이 가능
- 조인 결합 
    - 특정한 조건에 맞게 열을 결합 
    - 두개의 데이터프레임만 결합 가능
    - pandas에 내장된 merge() 함수를 이용
        - merge(데이터프레임명, 데이터프레임명)
        - on 매개변수 
            - 조건식(두 개의 데이터프레임이 공통적으로 가지고 있는 컬럼의 이름)
            - 컬럼의 이름이 다른 경우 
                - left_on, right_on 매개변수를 이용해서 결합이 가능
        - how 매개변수 
            - 두 개의 데이터프레임을 어떠한 방식으로 결합 
            - left : 왼쪽의 데이터프레임을 기준으로 열 결합 
            - right : 오른쪽의 데이터프레임을 기준으로 열 결합
            - inner : 두개의 데이터프레임이 공통적으로 가지고 있는 데이터들을 기준으로 결합 
            - outer : 두개의 데이터프레임을 합집합의 형태로 결합 

In [1]:
import pandas as pd

In [2]:
data = {
    'name' : ['test', 'test2', 'test3'], 
    'age' : [20, 30, 40]
}
df = pd.DataFrame(data)
df

,name,age
0,test,20
1,test2,30
2,test3,40


In [3]:
data2 = {
    'name' : ['test4', 'test5'], 
    'loc' : ['seoul', 'busan']
}
df2 = pd.DataFrame(data2)
df2

,name,loc
0,test4,seoul
1,test5,busan


In [4]:
# 단순한 행 결합 
pd.concat([df, df2], axis=0)
pd.concat([df, df2], ignore_index=True)
pd.concat([df, df2]).reset_index(drop=True)

,name,age,loc
0,test,20.0,NaN
1,test2,30.0,NaN
2,test3,40.0,NaN
3,test4,NaN,seoul
4,test5,NaN,busan


In [5]:
# 단순한 열 결합 
pd.concat([df, df2], axis=1)
pd.concat([df, df2], axis=1, ignore_index=True)

,0,1,2,3
0,test,20,test4,seoul
1,test2,30,test5,busan
2,test3,40,NaN,NaN


In [6]:
data3 = {
    'loc' : ['seoul', 'busan'], 
    'code' : ['11', '51']
}
df3 = pd.DataFrame(data3, index=[5, 6])
df3

,loc,code
5,seoul,11
6,busan,51


In [7]:
pd.concat([df, df2, df3], axis=0)

,name,age,loc,code
0,test,20.0,NaN,NaN
1,test2,30.0,NaN,NaN
2,test3,40.0,NaN,NaN
0,test4,NaN,seoul,NaN
1,test5,NaN,busan,NaN
5,NaN,NaN,seoul,11
6,NaN,NaN,busan,51


In [8]:
pd.concat([df, df2, df3], axis=1)

,name,age,name,loc,loc,code
0,test,20.0,test4,seoul,NaN,NaN
1,test2,30.0,test5,busan,NaN,NaN
2,test3,40.0,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,seoul,11
6,NaN,NaN,NaN,NaN,busan,51


In [9]:
data4 = {
    'id' : ['test', 'test2', 'test3'],
    'pass' : ['1234', '1111', '0000']
}
id_table = pd.DataFrame(data4)
id_table

,id,pass
0,test,1234
1,test2,1111
2,test3,0000


In [10]:
data5 = {
    'id' : ['test', 'test2'], 
    'item' : ['A', 'C']
}
id_item = pd.DataFrame(data5)
id_item

,id,item
0,test,A
1,test2,C


In [11]:
merge_data = pd.merge(
    id_table, id_item,
    on = 'id', 
    how = 'outer'
)

In [12]:
data6 = {
    'item_name' : ['A', 'B'], 
    'price' : [100, 200]
}
item_table = pd.DataFrame(data6)
item_table

,item_name,price
0,A,100
1,B,200


In [13]:
pd.merge(
    merge_data, item_table, 
    left_on = 'item', 
    right_on = 'item_name', 
    how='outer'
)

,id,pass,item,item_name,price
0,test,1234,A,A,100.0
1,test2,1111,C,NaN,NaN
2,test3,0000,NaN,NaN,NaN
3,NaN,NaN,NaN,B,200.0


In [14]:
merge_data.loc[2, 'item'] = 'A'

In [15]:
merge_data

,id,pass,item
0,test,1234,A
1,test2,1111,C
2,test3,0000,A


In [16]:
# item_table에 item_name이라는 컬럼의 이름을 item으로 변경
item_table.rename(
    columns = {
        'item_name' : 'item'
    }, inplace=True
)

In [17]:
pd.merge(
    merge_data, item_table, 
    on = 'item', 
    how = 'inner'
)

,id,pass,item,price
0,test,1234,A,100
1,test3,0000,A,100


- 특정 경로에 있는 파일의 목록을 가지고 와서 반복문을 이용하여 모든 데이터를 하나의 데이터프레임으로 결합
    1. 특정 경로에 있는 파일 목록 로드 
    2. 반복문을 이용하여 데이터를 하나씩 로드 
    3. 단순한 행 결합으로 데이터프레임을 결합 

In [18]:
import os 
from glob import glob

In [19]:
# os 라이브러리를 이용한 파일 목록 불러오기 
os.listdir('../csv/2017')

['201701_expense_list.csv',
 '201702_expense_list.csv',
 '201703_expense_list.csv',
 '201704_expense_list.csv',
 '201705_expense_list.csv',
 '201706_expense_list.csv',
 '201707_expense_list.csv',
 '201708_expense_list.csv',
 '201709_expense_list.csv',
 '201710_expense_list.csv',
 '201711_expense_list.csv',
 '201712_expense_list.csv']

In [20]:
# 특정 경로를 변수에 저장 
file_path = '../csv/2017/'
file_list = os.listdir(file_path)

In [21]:
file_list

['201701_expense_list.csv',
 '201702_expense_list.csv',
 '201703_expense_list.csv',
 '201704_expense_list.csv',
 '201705_expense_list.csv',
 '201706_expense_list.csv',
 '201707_expense_list.csv',
 '201708_expense_list.csv',
 '201709_expense_list.csv',
 '201710_expense_list.csv',
 '201711_expense_list.csv',
 '201712_expense_list.csv']

In [22]:
file_path + file_list[0]

'../csv/2017/201701_expense_list.csv'

In [23]:
pd.read_csv(file_path + file_list[0])

,nid,title,url,dept_nm_lvl_1,dept_nm_lvl_2,dept_nm_lvl_3,dept_nm_lvl_4,dept_nm_lvl_5,exec_yr,exec_month,expense_budget,expense_execution,category,dept_nm_full,exec_dt,exec_loc,exec_purpose,target_nm,payment_method,exec_amount
0,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-26 13:10,동해일식 (중구 무교동),기본소득과 장애인복지 논의간담회,장애인복지정책팀장 외 2명,카드,76000
1,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-25 22:41,김앤장 (중구 무교로),장애인단체 활동지원 논의간담회,장애인복지정책과장 외 3명,카드,102000
2,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-24 12:35,왕왕생고기 (중구 을지로),장애인 기본돌봄 복지시책 논의간담회,장애인복지정책팀장외7명,카드,80000
3,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-24 12:23,서울불고기 (중구 남대문로),서울시일자리통합지원센터 운영개선 논의간담회,장애인복지정책과장외5명,카드,112000
4,11430252,2017년 1월 장애인복지정책과 업무추진비 집행내역,http://opengov.seoul.go.kr/public/11430252,서울시본청,복지본부,장애인복지정책과,NaN,NaN,2017,1,NaN,NaN,NaN,복지본부 장애인복지정책과,2017-01-23 15:10,서울시청신매점,부서운영에 필요한 음료수 구매,장애인복지정책과직원,카드,16000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5512,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-11 13:01,차이79 종로구 새문안로5길,금융기관 보조금 지원계획수립관련 간담회 개최 비용지출,투자유치과장외5명,카드,27000
5513,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-06 12:58,강가 서울 중구 세종대로136,핀테크 산업지원시설 입주 관련 간담회 개최비용지출,금융협력관외6명,카드,70000
5514,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-04 12:47,이복만두 중구 무교동 27,F사 투자유치 협의를 위한 간담회 개최 비용지출,투자유치과장외4명,카드,57000
5515,15513286,2017년 1월 서울시본청_경제진흥본부_경제기획관_투자유치과 업무추진비 내역,http://opengov.seoul.go.kr/public/15513286,서울시본청,경제진흥본부,경제기획관,투자유치과,NaN,2017,1,NaN,NaN,NaN,경제진흥본부 투자유치과,2017-01-03 12:42,불고기브라더스 중구 태평로 1가,핀테크사업 전략수립을 위한 간담회 개최비용지출,경제진흥본부장외10명,카드,152400


In [24]:
# 2017년의 1-12월까지의 데이터를 하나의 데이터프레임으로 생성 

# 빈 데이터프레임을 생성 
df_2017 = pd.DataFrame()

# file_list를 기준으로 반복 실행하는 반복문을 생성 
for file in file_list:
    # file변수에 대입되는 값은 ? 파일의 이름
    # print(file)
    df = pd.read_csv(file_path + file)
    # df_2017에 df을 대입(단순한 행 결합을 하여 df_2017에 넣어준다.)
    df_2017 = pd.concat([df_2017, df])
    # 첫번째 반복 : file(201701_exepense_list.csv) -> read_csv(../csv/2017/201701_exepense_list.csv) -> df_2017(빈 데이터프레임)과 201701월 데이터가 단순 행 결합 -> df_2017에 대입
    # 두번째 반복 : file(201702_exepense_list.csv) -> read_csv(../csv/2017/201702_exepense_list.csv) -> df_2017(201701월 데이터)과 201702월 데이터가 단순 행 결합 -> df_2017에 대입

In [25]:
df_2017.info()

<class 'pandas.core.frame.DataFrame'>
Index: 70132 entries, 0 to 7285
Data columns (total 20 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   nid                70132 non-null  int64  
 1   title              70132 non-null  object 
 2   url                70132 non-null  object 
 3   dept_nm_lvl_1      70132 non-null  object 
 4   dept_nm_lvl_2      70074 non-null  object 
 5   dept_nm_lvl_3      61262 non-null  object 
 6   dept_nm_lvl_4      17939 non-null  object 
 7   dept_nm_lvl_5      3474 non-null   object 
 8   exec_yr            70132 non-null  int64  
 9   exec_month         70132 non-null  int64  
 10  expense_budget     3108 non-null   float64
 11  expense_execution  2805 non-null   float64
 12  category           1259 non-null   object 
 13  dept_nm_full       70053 non-null  object 
 14  exec_dt            70132 non-null  object 
 15  exec_loc           69360 non-null  object 
 16  exec_purpose       70110 non

In [32]:
# glob 라이브러리 
glob("../csv/2021/*.csv")

['../csv/2021\\202101_expense_list.csv',
 '../csv/2021\\202102_expense_list.csv',
 '../csv/2021\\202103_expense_list.csv',
 '../csv/2021\\202104_expense_list.csv',
 '../csv/2021\\202105_expense_list.csv',
 '../csv/2021\\202106_expense_list.csv',
 '../csv/2021\\202107_expense_list.csv',
 '../csv/2021\\202108_expense_list.csv',
 '../csv/2021\\202109_expense_list.csv',
 '../csv/2021\\202110_expense_list.csv',
 '../csv/2021\\202111_expense_list.csv',
 '../csv/2021\\202112_expense_list.csv']

In [27]:
# 지정된 경로에 파일들을 하나의 데이터프레임으로 결합하는 함수 
def data_load(_path, _end = 'csv'):
    # _path : 파일의 경로
    # _end : 파일의 확장자

    # 유저가 입력한 경로를 이용하여 파일의 목록을 로드 
    file_list = os.listdir(_path)
    # 혹시 유저가 경로의 마지막에 '/'가 없는 경우를 대비하여 마지막에 '/' 추가 
    file_path = _path + '/'

    # 비어있는 데이터프레임을 생성 
    result = pd.DataFrame()

    for file in file_list:
        # file -> 파일의 이름 
        # _end의 값에 따라 다른 함수를 이용
        if _end == 'csv':
            df = pd.read_csv(file_path + file)
        elif _end == 'json':
            df = pd.read_json(file_path + file)
        elif (_end == 'xlsx') | (_end =='xls'):
            df = pd.read_excel(file_path + file)
        elif _end == 'xml':
            df = pd.read_xml(file_path + file)
        else:
            return "지원하지 않은 확장자입니다."
        # 비어있는 데이터프레임에 단순 행 결합 
        result = pd.concat([result, df])
    print("데이터프레임 모두 결합 완료 크기는 ", len(result))
    return result

In [ ]:
data_load('../csv/2017')

In [ ]:
data_load("../csv/2019/", 'json')

In [ ]:
data_load("../csv/2020/", 'xlsx')

In [33]:
data_load('../csv/2021/', 'json')

ValueError: Expected object or value